# TATA STEEL MACHINE FAILURE PREDICTION



##### **Contribution**    - Ankita Dutta

# **Project Summary -**

This project focuses on predicting **machine failure** using a dataset containing sensor readings and failure records from industrial machines. The dataset includes numerical features such as **Air Temperature, Process Temperature, Rotational Speed, Torque, and Tool Wear**, along with binary indicators for different types of failures: **TWF (Tool Wear Failure), HDF (Heat Dissipation Failure), PWF (Power Failure), OSF (Overstrain Failure), and RNF (Random Failure).** The main objective is to build a machine learning model that can accurately predict whether a machine is likely to fail based on sensor readings, thereby enabling predictive maintenance and reducing operational downtime.  

### **Automating Data Loading via GitHub**  
Initially, dataset loading required manual uploads using **Google Colab's file import function**. To streamline the workflow, the dataset was instead **hosted on GitHub**, allowing it to be loaded directly using **Pandas' `read_csv()` function with a raw GitHub URL**. This eliminated the need for repeated manual uploads.

### **Extensive Exploratory Data Analysis (EDA)**  
Extensive **Exploratory Data Analysis (EDA)** was performed to understand the nature of the dataset and determine the best preprocessing steps. Various **visualization techniques** such as histograms, boxplots, correlation heatmaps, and pair plots were used to assess data distributions, detect outliers, and identify relationships between features. The **failure distribution was highly imbalanced**, with failures occurring in a very small percentage of cases. KDE (Kernel Density Estimation) plots and scatter plots helped visualize how different failure types were distributed across numerical features like torque and rotational speed. The insights gained from EDA played a crucial role in designing appropriate **preprocessing techniques** tailored to the dataset’s nature.  

### **Data Preprocessing and Feature Engineering**  
To prepare the data, unnecessary columns such as **ID, Product ID, and Type** were removed since they did not contribute to failure prediction. A new target variable, **Machine Failure**, was created by combining all individual failure types into a single binary column, where 1 indicates failure and 0 indicates normal operation. The dataset was **normalized** using **StandardScaler** to ensure that all features contributed equally to the model’s learning process, preventing bias from larger numerical values.  

### **Handling Class Imbalance with SMOTE**  
Given that failures were underrepresented in the dataset, **SMOTE (Synthetic Minority Over-sampling Technique)** was applied to artificially increase the number of failure cases. Initially, aggressive SMOTE application resulted in overfitting, where the model achieved near-perfect accuracy on training data but performed poorly on unseen test data. To counter this, **a less aggressive SMOTE approach** was implemented, balancing the failure cases while avoiding excessive duplication of synthetic data. The resampling effect was visualized using KDE plots to ensure that the feature distributions remained realistic after applying SMOTE.  

### **Model Selection and Evaluation**  
Various machine learning models were tested, including **Logistic Regression, Random Forest, and XGBoost**, to determine the most suitable approach for failure prediction. While **Random Forest and XGBoost** initially showed higher accuracy, they also exhibited signs of overfitting, failing to generalize well to test data. **Logistic Regression** was chosen as a more stable alternative, as it provided a balance between accuracy and generalization, reducing the risk of misleadingly high training performance. The final model was trained with **L2 regularization (Ridge regression)** to further prevent overfitting.  

The models were evaluated using multiple performance metrics, including **accuracy, precision, recall, F1-score, confusion matrix, and ROC-AUC curve** to assess their effectiveness. The **confusion matrix** provided insight into the distribution of false positives and false negatives, which is crucial for failure prediction, as false negatives (missed failures) could lead to costly machine breakdowns. **Precision and recall trade-offs** were analyzed to ensure that the model correctly identified failures without excessively flagging normal operations. The **ROC-AUC curve** measured how well the model distinguished between failed and non-failed machines, ensuring robust decision-making.  

### **Final Results and Insights**  
The final model achieved a balanced **accuracy of approximately 99.82%**, meaning it successfully identified most machine failures in the test set. The **precision for failure cases was 89%**, indicating that most failure predictions were correct, with a small number of false positives. **Macro and weighted F1-scores** confirmed strong overall performance. The final model selection prioritized avoiding overfitting while maintaining a high ability to predict failures accurately.  



# **GitHub Link -**

# **Problem Statement**


Develop a predictive maintenance model to anticipate machine failures in TATA Steel’s manufacturing process, minimizing downtime and optimizing maintenance efficiency.

## ***1. About Data***

### Import Libraries

In [ ]:
!pip install dask[dataframe]

In [ ]:
# Import Libraries
# Import Libraries
import os
import pandas as pd  # data manipulation & preprocessing
import numpy as np  # numerical calculations
from scipy import stats # mathematical & statistical computations

# ML Libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Gradient Boosting Libraries
import xgboost as xgb  # XGBoost
import lightgbm as lgb  # LightGBM

In [ ]:
'''from google.colab import files
uploaded = files.upload()'''


In [ ]:
'''from google.colab import files
uploaded = files.upload()'''


In [ ]:
import pandas as pd

# Load train dataset directly from GitHub
train_url = "https://raw.githubusercontent.com/ankitaXdutta/TataMachineFailure/main/train.csv"
train_df = pd.read_csv(train_url)

# Load test dataset
test_url = "https://raw.githubusercontent.com/ankitaXdutta/TataMachineFailure/main/test.csv"
test_df = pd.read_csv(test_url)

# Check if the data loaded correctly
train_df.head()


### Dataset Loading

In [ ]:
import pandas as pd

# Use direct GitHub raw links
train_url = "https://raw.githubusercontent.com/ankitaXdutta/TataMachineFailure/main/train.csv"
test_url = "https://raw.githubusercontent.com/ankitaXdutta/TataMachineFailure/main/test.csv"

# Load datasets from GitHub
train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

# Verify the first few rows
train_df.head(), test_df.head()


### Dataset First View

In [ ]:
# Dataset First Look
print("Train Data:")
print(train_df.head())
print("\nTest Data:")
print(test_df.head())

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
print("\nTrain Data Shape:", train_df.shape)
print("Test Data Shape:", test_df.shape)

### Dataset Information

In [ ]:
# Dataset Info
print("\nTrain Data Info:")
print(train_df.info())
print("\nTest Data Info:")
print(test_df.info())

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print("\nDuplicate values in Train Data:", train_df.duplicated().sum())
print("Duplicate values in Test Data:", test_df.duplicated().sum())

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
print("\nMissing values in Train Data:")
print(train_df.isnull().sum())
print("\nMissing values in Test Data:")
print(test_df.isnull().sum())

In [ ]:
plt.figure(figsize=(12, 6))

# Use a more contrasting colormap and include annotations
sns.heatmap(train_df.isnull(),
            cmap="coolwarm",
            cbar=True,
            linewidths=0.5,
            linecolor='black')

plt.title("Missing Values in Train Data", fontsize=16, fontweight='bold')
plt.xlabel("Features", fontsize=12)
plt.ylabel("Samples", fontsize=12)
plt.xticks(rotation=45)
plt.yticks([])  # Hide y-axis labels for better clarity
plt.suptitle("White areas indicate missing values (if any)", fontsize=10, color='gray')

plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

# Use a more contrasting colormap and include annotations
sns.heatmap(test_df.isnull(),
            cmap="coolwarm",
            cbar=True,
            linewidths=0.5,
            linecolor='black')

plt.title("Missing Values in Test Data", fontsize=16, fontweight='bold')
plt.xlabel("Features", fontsize=12)
plt.ylabel("Samples", fontsize=12)
plt.xticks(rotation=45)
plt.yticks([])  # Hide y-axis labels for better clarity
plt.suptitle("White areas indicate missing values (if any)", fontsize=10, color='gray')

plt.show()


### What did you know about your dataset?

The dataset consists of 136,429 training samples and 90,954 test samples, with 14 columns in the train set and 13 in the test set (missing the "Machine failure" column). It includes sensor readings like air temperature, process temperature, rotational speed, torque, and tool wear, along with failure labels (TWF, HDF, PWF, OSF, RNF). The data has no missing or duplicate values, making it clean and ready for analysis. The goal is to predict machine failure based on these features.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
print("\nTrain Data Columns:")
print(train_df.columns)
print("\nTest Data Columns:")
print(test_df.columns)

In [ ]:
# Dataset Describe
print("\nTrain Data Description:")
print(train_df.describe())
print("\nTest Data Description:")
print(test_df.describe())

### Variables Description

We are taking a look at count, mean, std, min, 25%, 50%, 75% and max.
The dataset contains multiple variables describing machine performance and failures.

The ID is a unique identifier with a mean of 68,214, ranging from 0 to 136,428, and is evenly distributed.

Air temperature [K], which measures ambient air temperature, has a mean of 299.86 K, ranging from 295.3 K to 304.4 K, with a standard deviation of 1.86.

Process temperature [K], representing the machine’s internal temperature, has a mean of 309.94 K, with values between 305.8 K and 313.8 K and a standard deviation of 1.38.

Rotational speed [rpm] measures the machine’s speed, averaging 1520.33 rpm, with a minimum of 1181 rpm and a maximum of 2886 rpm, and a standard deviation of 138.73.

Torque [Nm], indicating the rotational force, has a mean of 40.35 Nm, ranging from 3.8 Nm to 76.6 Nm, with a standard deviation of 8.50.

Tool wear [min], which tracks tool wear time, has an average of 104.41 minutes, with values spanning from 0 to 253 minutes and a standard deviation of 63.97.

Failures are recorded as binary indicators, where Machine failure occurs in 1.57% of cases (mean = 0.0157), with a minimum of 0 and a maximum of 1.









### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
# Check Unique Values for each variable in Train Data
print("\nUnique Values in Train Data:")
for column in train_df.columns:
    print(f"{column}: {train_df[column].nunique()} unique values")

# Check Unique Values for each variable in Test Data
print("\nUnique Values in Test Data:")
for column in test_df.columns:
    print(f"{column}: {test_df[column].nunique()} unique values")

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
train_df.info()
train_df.describe()
train_df.head() # we can drop id and product id, and label encode type

In [ ]:
train_df.isnull().sum()
train_df.duplicated().sum()
train_df.nunique() #so there are no wrangling methods needed

In [ ]:
train_df["Machine failure"].value_counts()
# heavy class imbalance, we will address it later in pre-processing section

In [ ]:
train_df.select_dtypes(include="object").head() #categorical columns
train_df["Type"].value_counts() #categorical features

In [ ]:
# Write your code to make your dataset analysis ready.
from sklearn.preprocessing import LabelEncoder

# Drop unnecessary columns in both train and test
train_df.drop(columns=["id", "Product ID"], inplace=True)
test_df.drop(columns=["id", "Product ID"], inplace=True)

# Apply Label Encoding to "Type" in both train and test
label_encoder = LabelEncoder()
train_df["Type"] = label_encoder.fit_transform(train_df["Type"])
test_df["Type"] = label_encoder.transform(test_df["Type"])  # Use same encoder to avoid mismatch

# Display results
train_df.head(), test_df.head()

### What all manipulations have you done and insights you found?

The following manipulations were performed on the dataset:

The "id" and "Product ID" columns were dropped from both train and test sets as they were unnecessary for analysis.

The "Type" column, which contained categorical data, was converted into numerical values using Label Encoding to facilitate machine learning models.

The dataset structure was preserved, and no missing values or outliers were explicitly handled in this step.

Insights from the processed dataset include the retention of key numerical features, with no apparent changes to distributions beyond encoding, ensuring consistency between train and test data.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code
plt.figure(figsize=(10,5))
sns.histplot(train_df["Air temperature [K]"], kde=True, bins=30, color="blue")
plt.title("Distribution of Air Temperature [K]", fontsize=14)
plt.xlabel("Temperature (K)")
plt.ylabel("Count")
plt.show()



##### 1. Why did you pick the specific chart?

A histogram was chosen to analyze the distribution of air temperature. This visualization is effective for identifying patterns, frequency distributions, and potential anomalies in temperature data. The density curve further helps in understanding underlying trends.

##### 2. What is/are the insight(s) found from the chart?

The temperature distribution is multimodal, meaning there are multiple peaks, suggesting different operational or environmental conditions affecting temperature.

Most temperatures range between 296K and 304K, with certain temperature values occurring more frequently.

The presence of multiple peaks may indicate variations due to different time periods (e.g., day vs. night) or environmental factors.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, these insights can be beneficial for:

Optimizing climate control systems by understanding typical temperature ranges and adjusting HVAC operations accordingly.

Predicting equipment performance in different temperature conditions, helping with preventive maintenance.

Identifying anomalies that could indicate potential operational issues, allowing for proactive interventions.

By leveraging these insights, businesses can improve efficiency, reduce energy costs, and ensure stable operational conditions.

However, if these temperature variations are uncontrolled, they could result in inconsistent machine performance or product defects, potentially leading to negative business impacts.

#### Chart - 2

In [ ]:
# Chart - 2 visualization code
plt.figure(figsize=(8,6))
sns.scatterplot(x=train_df["Air temperature [K]"], y=train_df["Process temperature [K]"], alpha=0.5)
plt.title("Air Temperature vs Process Temperature", fontsize=14)
plt.xlabel("Air Temperature (K)")
plt.ylabel("Process Temperature (K)")
plt.show()


##### 1. Why did you pick the specific chart?

The scatter plot was chosen to analyze the relationship between air temperature and process temperature. It helps visualize correlations, patterns, and potential dependencies between these two variables, which are critical in process optimization.

##### 2. What is/are the insight(s) found from the chart?

There is a clear positive correlation between air temperature and process temperature, meaning an increase in air temperature tends to increase process temperature.

The data points form a dense, structured pattern, suggesting a strong, predictable relationship.

There are some outliers where process temperature deviates from the trend, which may indicate anomalies or inefficiencies in the process.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the insights can help businesses optimize their process control by predicting process temperature based on air temperature. This can improve efficiency, reduce energy costs, and minimize potential equipment failures due to temperature fluctuations.

However, if the relationship is not controlled properly, unexpected deviations could lead to negative outcomes, such as product defects or equipment malfunctions.

#### Chart - 3

In [ ]:
# Chart - 3 visualization code
plt.figure(figsize=(10,5))
sns.histplot(train_df["Rotational speed [rpm]"], kde=True, bins=40, color="red")
plt.title("Rotational Speed Distribution", fontsize=14)
plt.xlabel("Rotational Speed (rpm)")
plt.ylabel("Count")
plt.show()


##### 1. Why did you pick the specific chart?

The histogram was chosen to analyze the distribution of rotational speed (rpm) across the dataset. It helps identify the most common operating speeds, detect outliers, and understand variability in the system.

##### 2. What is/are the insight(s) found from the chart?

The distribution is right-skewed, with most data points concentrated around 1400–1600 rpm.

There is a peak near 1500 rpm, indicating that the system most frequently operates around this speed.

A long tail extends towards higher rpm values, suggesting occasional higher-speed operations, which might indicate anomalies, inefficiencies, or specific operational needs.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, these insights can lead to positive business impact by:

Helping optimize machine performance by maintaining operation within the most efficient speed range.

Identifying outliers or unexpected high-speed occurrences that may require further investigation to prevent potential failures or inefficiencies.

Reducing maintenance costs by ensuring that rotational speed remains within a safe and efficient range.

However, if high-speed operations lead to increased wear and tear, it could negatively impact the business unless properly managed.

#### Chart - 4

In [ ]:
# Chart - 4 visualization code
plt.figure(figsize=(8,6))
sns.scatterplot(x=train_df["Rotational speed [rpm]"], y=train_df["Torque [Nm]"], alpha=0.5)
plt.title("Torque vs Rotational Speed", fontsize=14)
plt.xlabel("Rotational Speed (rpm)")
plt.ylabel("Torque (Nm)")
plt.show()


##### 1. Why did you pick the specific chart?

A scatter plot was chosen to examine the relationship between rotational speed (rpm) and torque (Nm). This type of visualization helps in identifying trends, correlations, and potential operational inefficiencies.

##### 2. What is/are the insight(s) found from the chart?

There is an inverse relationship between rotational speed and torque, where higher speeds generally correspond to lower torque values.

Most data points cluster at lower speeds with higher torque, indicating that the system operates in that range more frequently.

There are some outliers with high torque at higher speeds, which might suggest operational anomalies or inefficiencies.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, these insights can drive a positive business impact by:

Allowing for process optimization by ensuring operations stay within the most efficient speed-torque range.

Identifying unusual behavior that could indicate mechanical stress or inefficiencies, preventing costly maintenance or downtime.

Helping adjust control parameters for better energy efficiency and performance.
However, if high torque at lower speeds leads to excessive wear or energy consumption, it could negatively impact costs unless properly managed.

#### Chart - 5

In [ ]:
# Chart - 5 visualization code
plt.figure(figsize=(8, 6))
sns.boxplot(x="Machine failure", y="Torque [Nm]", data=train_df, hue="Machine failure", legend=False, palette="coolwarm")
plt.title("Distribution of Torque by Machine Failure")
plt.xlabel("Machine Failure (0 = No, 1 = Yes)")
plt.ylabel("Torque [Nm]")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()



##### 1. Why did you pick the specific chart?

A box plot was chosen to compare the distribution of torque values between machines that failed and those that did not. This type of visualization effectively highlights key statistics such as median, interquartile range, and outliers, making it useful for identifying patterns related to machine failure.

##### 2. What is/are the insight(s) found from the chart?

Machines that failed generally had higher torque values compared to those that did not fail.

The median torque for failed machines is significantly higher than for non-failed machines, suggesting a correlation between increased torque and failure rates.

Failed machines exhibit a wider distribution of torque values, including extreme outliers at both high and low torque levels, which may indicate operational stress or irregular conditions leading to failure.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, these insights can positively impact the business by:

Improving maintenance schedules: Identifying high torque as a failure risk factor allows for preventive maintenance and early intervention.

Enhancing machine design: Engineers can optimize torque limits and introduce safeguards to prevent excessive stress on components.

Reducing downtime: Predicting failures based on torque distribution helps minimize unplanned outages and increases operational efficiency.


However, there are potential negative insights:

Higher failure rates could indicate design flaws: If machines are consistently failing at higher torques, it may require costly redesigns or more robust components.

Increased maintenance costs: If torque monitoring leads to more frequent interventions, businesses may face higher short-term expenses.

#### Chart - 6

In [ ]:
# Chart - 6 visualization code
plt.figure(figsize=(8, 6))
sns.boxplot(x="Machine failure", y="Tool wear [min]", data=train_df, hue="Machine failure", legend=False, palette="viridis")
plt.title("Tool Wear vs Machine Failure", fontsize=14)
plt.xlabel("Machine Failure (0 = No, 1 = Yes)")
plt.ylabel("Tool Wear (minutes)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


##### 1. Why did you pick the specific chart?

A box plot was chosen to compare tool wear time between machines that failed and those that did not. This type of visualization effectively highlights the distribution, median values, interquartile ranges, and potential outliers, helping to understand whether tool wear is a factor in machine failure.

##### 2. What is/are the insight(s) found from the chart?

Machines that failed generally had higher tool wear times compared to non-failed machines.

The median tool wear time for failed machines is significantly higher than for non-failed machines, suggesting a correlation between increased tool wear and machine failure.

The distributions for both failed and non-failed machines are similar in range, but failed machines tend to cluster more toward higher wear times.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, these insights can positively impact the business by:

Optimizing maintenance schedules: Identifying tool wear as a failure risk factor enables proactive maintenance before failure occurs.

Improving operational efficiency: Monitoring tool wear trends can help reduce unplanned downtime, leading to better productivity.

Extending machine lifespan: Adjusting maintenance and operational parameters based on wear data can reduce premature machine failures.

Potential negative insights:

Higher maintenance costs: Frequent tool replacements or interventions may increase short-term costs.

Potential design flaws: If excessive tool wear leads to failure, redesigning tools or processes may be necessary, which can be costly.

#### Chart - 7

In [ ]:
# Count occurrences of each failure type
failure_types = ["TWF", "HDF", "PWF", "OSF", "RNF"]
fail_counts = [train_df[f].sum() for f in failure_types]

plt.figure(figsize=(10, 5))
sns.barplot(x=failure_types, y=fail_counts, palette="magma")

# Add value labels on top of bars
for i, count in enumerate(fail_counts):
    plt.text(i, count + 10, str(count), ha='center', fontsize=12, fontweight='bold')

plt.title("Failure Counts by Type", fontsize=14)
plt.xlabel("Failure Type", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.grid(axis='y', linestyle="--", alpha=0.7)  # Light grid for better readability
plt.show()


##### 1. Why did you pick the specific chart?

A bar chart was chosen because it is ideal for displaying categorical data and comparing failure counts across different types. It effectively highlights which failure types are more frequent, making it easy to identify key problem areas in machine performance.

##### 2. What is/are the insight(s) found from the chart?

HDF (Heat Dissipation Failure) is the most common failure type, occurring 704 times, making it a major area of concern.

OSF (Overstrain Failure) follows closely with 540 occurrences, indicating another significant issue.

TWF (Tool Wear Failure) is the least frequent with 212 occurrences, suggesting it may not be the primary cause of breakdowns.

The variation in failure counts suggests that some issues (like HDF and OSF) are more critical and require more attention than others.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, these insights can help create a positive business impact by:

Prioritizing maintenance efforts: Since HDF and OSF are the leading failure types, businesses can allocate more resources to mitigate these issues.

Reducing downtime: Addressing common failure causes proactively can improve machine uptime and efficiency.

Enhancing product design: If HDF is the leading cause, better heat management solutions should be developed to prevent failures.

Potential negative impact:

Increased short-term costs: Implementing new maintenance strategies or redesigning components may require additional investment.

Resource reallocation challenges: Focusing on high-frequency failures may divert attention from less frequent but equally damaging issues.

#### Chart - 8

In [ ]:
# Chart - 8 visualization code
plt.figure(figsize=(8,6))
sns.violinplot(x=train_df["Machine failure"], y=train_df["Torque [Nm]"], palette="coolwarm")
plt.title("Torque Distribution by Machine Failure", fontsize=14)
plt.xlabel("Machine Failure (0 = No, 1 = Yes)")
plt.ylabel("Torque (Nm)")
plt.show()


##### 1. Why did you pick the specific chart?

A violin plot was chosen because it effectively displays the distribution, density, and spread of torque values for machines that failed (1) versus those that did not (0). Unlike a boxplot, a violin plot provides insight into how torque values are concentrated, helping to identify key patterns and deviations in torque distribution related to failures.

##### 2. What is/are the insight(s) found from the chart?

Machines that failed (1) tend to have higher torque values on average compared to machines that did not fail.

The spread of torque values is wider for failed machines, indicating more variability in torque conditions leading to failure.

Machines that did not fail (0) have a more concentrated torque distribution, suggesting they operate within a more controlled torque range.

Higher torque values (above ~50 Nm) are more frequent among failed machines, indicating a possible threshold where torque significantly contributes to failures.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, these insights can lead to a positive business impact by:

Setting torque thresholds: If high torque is a major contributor to failures, machine settings can be adjusted to maintain torque within a safer range.

Preventive maintenance: Machines experiencing excessive torque variations can be flagged for early maintenance before failure occurs.

Reducing downtime: By controlling torque levels, unexpected failures can be minimized, improving machine efficiency and production reliability.

However,
Reduced operational flexibility - Strict torque limitations might affect machine performance in scenarios where higher torque is necessary.

#### Chart - 9

In [ ]:
plt.figure(figsize=(8,6))
sns.kdeplot(train_df.loc[train_df["Machine failure"] == 0, "Rotational speed [rpm]"], label="No Failure", fill=True, alpha=0.5)
sns.kdeplot(train_df.loc[train_df["Machine failure"] == 1, "Rotational speed [rpm]"], label="Failure", fill=True, alpha=0.5)
plt.title("Density Distribution of Rotational Speed by Machine Failure", fontsize=14)
plt.xlabel("Rotational Speed (rpm)")
plt.ylabel("Density")
plt.legend()
plt.show()


##### 1. Why did you pick the specific chart?

A density plot was chosen because it effectively shows the distribution of rotational speeds for machines that failed versus those that did not. This allows for a clear comparison of how rotational speed varies between both categories, helping to identify potential risk zones for machine failures.

##### 2. What is/are the insight(s) found from the chart?

Machines that failed (orange) tend to have lower rotational speeds, peaking around 1300-1400 rpm.
Machines that did not fail (blue) have a slightly higher peak and a wider distribution, extending to around 2000+ rpm.
There are very few failures beyond 2000 rpm, suggesting that failures are more likely at lower speeds.
Overlap exists between 1300-1500 rpm, indicating that some non-failing machines also operate within this range but possibly under different conditions.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, these insights can have a positive business impact by:

Identifying high-risk speed ranges: Since failures occur more frequently at lower speeds (~1300-1400 rpm), maintenance teams can monitor machines operating in this range more closely.

Optimizing machine settings: Adjusting operational speeds to avoid failure-prone zones might improve machine longevity.

Predictive maintenance: Using this data, machine learning models can predict failures based on rotational speed patterns, reducing downtime.

However there may be Operational limitations: If machines are required to run at lower speeds due to external constraints (e.g., load balancing), avoiding this range might not always be feasible.
There may be false positives in failure prediction as some machines operating at ~1300-1500 rpm do not fail, so an overly strict response might lead to unnecessary maintenance costs.

#### Chart - 10

In [ ]:
# Chart - 10 visualization code
plt.figure(figsize=(8,6))
sns.stripplot(x="Machine failure", y="Air temperature [K]", data=train_df, jitter=True, alpha=0.5, hue="Machine failure", palette="viridis", legend=False)
plt.title("Air Temperature vs Machine Failure", fontsize=14)
plt.xlabel("Machine Failure (0 = No, 1 = Yes)")
plt.ylabel("Air Temperature (K)")
plt.show()



##### 1. Why did you pick the specific chart?

A strip plot was chosen to compare air temperature distributions for machines that failed (1) and those that did not fail (0). This helps in visually identifying whether air temperature has a significant correlation with machine failures.

##### 2. What is/are the insight(s) found from the chart?

The temperature range for both failed and non-failed machines appears quite similar, between 295K and 305K.
There is no clear separation between the two categories, indicating that air temperature alone may not be a strong predictor of failure.
If there is a slight trend, it might suggest that failures happen more frequently at higher temperatures, but the overlap makes this inconclusive.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Positive Impact: The chart suggests that air temperature alone is not a strong failure predictor, preventing unnecessary temperature-based interventions. Businesses can focus on more impactful factors like rotational speed, vibration, or pressure.

Negative Impact: If misinterpreted, businesses might ignore temperature monitoring entirely, even though it could have a combined effect with other variables. Multivariate analysis should be done to determine if temperature contributes to failures when combined with other factors.

#### Chart - 11

In [ ]:
# Chart - 11 visualization code
sns.jointplot(x=train_df["Process temperature [K]"], y=train_df["Rotational speed [rpm]"], kind="hex", cmap="coolwarm")
plt.suptitle("Process Temperature vs. Rotational Speed", fontsize=14)
plt.show()

##### 1. Why did you pick the specific chart?

A jointplot was chosen because :
It shows the relationship between process temperature and rotational speed effectively. The hexbin representation helps visualize data density, highlighting common operational ranges. The marginal histograms provide extra insights into the individual distributions of both variables. This type of plot is useful for identifying clusters, trends, and potential anomalies in machine behavior.

##### 2. What is/are the insight(s) found from the chart?

The majority of operations occur in the 308–312 K range for process temperature and 1400–1600 rpm for rotational speed.

There are a few high-density clusters (red areas), meaning certain process conditions are more frequent.

Higher rotational speeds (>2000 rpm) are rare, which might indicate operational constraints or efficiency limitations.

The marginal histograms show a right-skewed distribution for rotational speed, meaning lower speeds are more common.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Positive Impact:

Helps optimize machine performance by identifying the most stable operating conditions.

Can assist in predictive maintenance, as deviations from these common values may signal potential failures.

Provides a data-driven approach to efficiency improvements by focusing on the most frequent conditions.

Negative Impact:

If the company assumes only the most frequent conditions matter, rare but critical failure points may be ignored. Potential underutilization of machines if higher rotational speeds are avoided due to limited data.

#### Chart - 12

In [ ]:
# Chart - 12 visualization code
plt.figure(figsize=(10,5))
sns.countplot(data=train_df, x="Machine failure", hue="Type", palette="magma")
plt.title("Failure Distribution by Machine Type", fontsize=14)
plt.xlabel("Machine Failure (0 = No, 1 = Yes)")
plt.ylabel("Count")
plt.legend(title="Type")
plt.show()


##### 1. Why did you pick the specific chart?

The countplot was chosen because it effectively visualizes categorical data, making it easy to compare machine failure occurrences across different types. It provides a clear and simple representation of how failures are distributed among machine types, allowing for quick insights.

##### 2. What is/are the insight(s) found from the chart?

Machine Type 1 has the highest number of machines and dominates the dataset.
Machine Type 2 has significantly fewer machines but still contributes to failures.
Failures are much lower in number compared to non-failures for all machine types.
Some machine types might have a higher failure rate relative to their total count, which requires deeper investigation.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

The gained insights can create a positive business impact by helping identify which machine types are more prone to failure, enabling better maintenance strategies and resource allocation.

However, relying solely on total failure counts without considering failure rates could lead to inefficient decision-making,

#### Chart - 13

In [ ]:
# Chart - 13 visualization code
import numpy as np
import seaborn as sns

plt.figure(figsize=(8,6))
sns.regplot(x=train_df["Tool wear [min]"], y=train_df["Machine failure"], scatter_kws={"alpha": 0.3}, order=2, line_kws={"color": "red"})
plt.title("Polynomial Regression: Tool Wear vs Machine Failure", fontsize=14)
plt.xlabel("Tool Wear (minutes)")
plt.ylabel("Machine Failure Probability")
plt.show()


##### 1. Why did you pick the specific chart?

The polynomial regression chart was chosen because it effectively visualizes the non-linear relationship between tool wear and machine failure probability, helping to identify trends beyond a simple linear pattern. The red curve highlights how failure probability changes as tool wear increases.

##### 2. What is/are the insight(s) found from the chart?

Machine failure probability remains low for most of the tool wear range.

A slight increase in failure probability is observed at high tool wear levels.

Failures are scattered at both extremes, indicating that wear alone may not be the sole cause.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

These insights can positively impact business by optimizing maintenance schedules to prevent failures before tool wear reaches critical levels, reducing unexpected downtime.

However, if businesses rely solely on tool wear as a failure predictor without considering other factors, they might miss underlying issues, potentially leading to increased machine breakdowns and operational inefficiencies.

In [ ]:
# Check for non-numeric columns
non_numeric_cols = train_df.select_dtypes(exclude=['number']).columns
print("Non-numeric columns:", non_numeric_cols)


#### Chart - 14 - Correlation Heatmap

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select only numeric columns
numeric_cols = train_df.select_dtypes(include=['number'])

# Compute correlation matrix
corr_matrix = numeric_cols.corr()

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5, cbar=True)
plt.title("Feature Correlation Heatmap", fontsize=14)
plt.show()


##### 1. Why did you pick the specific chart?

The heatmap was chosen because it clearly visualizes the correlation between different features, making it easier to identify strong relationships and potential dependencies between variables affecting machine failure. The color gradient helps in quickly spotting significant correlations.

##### 2. What is/are the insight(s) found from the chart?

Air and process temperature show a strong correlation (0.86), indicating they increase together.

Rotational speed and torque have a strong negative correlation (-0.78), suggesting that higher speeds lead to lower torque.

Machine failure has moderate correlations with TWF (0.31), HDF (0.56), and OSF (0.49), indicating multiple factors contribute to failures rather than just tool wear.

Tool wear has a very weak correlation with machine failure (0.06), meaning wear alone is not a strong predictor of failure.

Process temperature and air temperature have high correlation (0.86), meaning changes in one likely affect the other.

HDF (Heat Dissipation Failure) has the highest correlation with machine failure (0.56), suggesting overheating plays a significant role in breakdowns.

OSF (Overstrain Failure) also has a strong correlation with machine failure (0.49), indicating mechanical stress is another key contributor.

Rotational speed and torque have a strong inverse relationship (-0.78), implying that higher speeds require less torque, likely due to mechanical design constraints.

Other failure modes like PWF (Power Failure) and RNF (Random Failure) have weaker correlations, indicating they are less predictable from the given features.








#### Chart - 15 - Pair Plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Selecting key numerical features
pairplot_features = ["Air temperature [K]", "Process temperature [K]",
                     "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]

# Sample only a fraction of data to speed up plotting (adjust sample size as needed)
sample_df = train_df.sample(n=500, random_state=42)  # Adjust sample size if necessary

# Pair plot with sampled data
sns.pairplot(sample_df, vars=pairplot_features, hue="Machine failure", palette="coolwarm", diag_kind="kde")

plt.suptitle("Pair Plot of Key Features (Sampled)", y=1.02, fontsize=14)
plt.show()



##### 1. Why did you pick the specific chart?

This pair plot allows visualization of relationships between multiple variables simultaneously, helping identify patterns, correlations, and clusters associated with machine failure. It is useful for detecting trends and potential failure indicators.

##### 2. What is/are the insight(s) found from the chart?

Process temperature and air temperature show a strong linear correlation, indicating that changes in one affect the other.

Torque and rotational speed appear to have an inverse relationship, which is expected in mechanical systems.

Machine failures (orange dots) are scattered across different regions, suggesting failures do not depend on a single variable but multiple factors.

Tool wear distribution shows a concentration of data points around mid-range values, indicating that extreme wear levels are less common.

Failures are more frequent in certain torque and rotational speed ranges, indicating potential thresholds for risk.

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Hypothesis 1 : Rotational speed significantly differs between machines that fail and machines that do not fail

Null Hypothesis (H₀): The rotational speed distribution is the same for failing and non-failing machines.

Alternate Hypothesis (H₁): The rotational speed distribution is significantly different between failing and non-failing machines.

In [ ]:
#Since the dataset has 134,281 entries, Shapiro-Wilk is unreliable (N > 5000).
#Instead, we check normality using Kolmogorov-Smirnov

import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Extract rotational speed for both groups
failures = train_df[train_df["Machine failure"] == 1]["Rotational speed [rpm]"]
non_failures = train_df[train_df["Machine failure"] == 0]["Rotational speed [rpm]"]

# Kolmogorov-Smirnov Test (for large datasets)
ks_stat_fail, p_fail = stats.kstest(failures, 'norm', args=(failures.mean(), failures.std()))
ks_stat_non_fail, p_non_fail = stats.kstest(non_failures, 'norm', args=(non_failures.mean(), non_failures.std()))

print(f"Kolmogorov-Smirnov Test P-Value (Failures): {p_fail}")
print(f"Kolmogorov-Smirnov Test P-Value (Non-Failures): {p_non_fail}")

# Q-Q Plot (visual check)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
stats.probplot(failures, dist="norm", plot=plt)
plt.title("Q-Q Plot - Failures")

plt.subplot(1, 2, 2)
stats.probplot(non_failures, dist="norm", plot=plt)
plt.title("Q-Q Plot - Non-Failures")

plt.show()


P-value for failures: 1.67e-162

P-value for non-failures: 0.0

Since both p-values are extremely low (p < 0.05), we reject the assumption of normality.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value
import scipy.stats as stats

# Splitting the data into two groups
fail_speed = train_df.loc[train_df["Machine failure"] == 1, "Rotational speed [rpm]"]
no_fail_speed = train_df.loc[train_df["Machine failure"] == 0, "Rotational speed [rpm]"]

# Checking normality
stat_fail, p_fail = stats.shapiro(fail_speed)
stat_no_fail, p_no_fail = stats.shapiro(no_fail_speed)

# If data is normal, use independent t-test; otherwise, use Mann-Whitney U test
if p_fail > 0.05 and p_no_fail > 0.05:
    stat, p_value = stats.ttest_ind(fail_speed, no_fail_speed, equal_var=False)  # Welch's t-test
    test_used = "Welch’s t-test (for unequal variances)" #wont be used anyway added to remove renundencies
else:
    stat, p_value = stats.mannwhitneyu(fail_speed, no_fail_speed, alternative="two-sided")
    test_used = "Mann-Whitney U test (for non-normal data)"

print(f"Statistical Test Used: {test_used}")
print(f"P-Value: {p_value}")

# Conclusion
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis (H₀) - There is a significant difference in rotational speed between failing and non-failing machines.")
else:
    print("Fail to reject the null hypothesis (H₀) - No significant difference in rotational speed between failing and non-failing machines.")


##### Which statistical test have you done to obtain P-Value?

Mann-Whitney U test (as data is non-normal).


##### Why did you choose the specific statistical test?



Mann-Whitney U test is used when the data does not follow a normal distribution and is non-parametric.

So, P-Value: 0.0 is very small, meaning a strong difference

Decision to take : Rejecting the null hypothesis (H₀).

Conclusion: There is a significant difference in rotational speed between machines that fail and those that do not fail.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Hypothesis 2 : Torque values differs between machines that fail and machines that do not fail

Null Hypothesis (H₀): The torque values are similar between machines that fail and those that do not.

Alternative Hypothesis (H₁): The torque values significantly differ between failing and non-failing machines.

#### 2. Perform an appropriate statistical test.

In [ ]:
from scipy.stats import mannwhitneyu

# Splitting data into two groups: failed and non-failed machines
failures = train_df[train_df['Machine failure'] == 1]['Torque [Nm]']
non_failures = train_df[train_df['Machine failure'] == 0]['Torque [Nm]']

# Perform Mann-Whitney U Test
stat, p_value = mannwhitneyu(failures, non_failures, alternative='two-sided')

print(f"Mann-Whitney U Test P-Value: {p_value}")

# Conclusion
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis (H₀) - There is a significant difference in torque between failing and non-failing machines.")
else:
    print("Fail to reject the null hypothesis (H₀) - No significant difference in torque between failing and non-failing machines.")


##### Which statistical test have you done to obtain P-Value?

Mann-Whitney U test

##### Why did you choose the specific statistical test?

The torque values are continuous, and previous normality tests (Shapiro-Wilk/Kolmogorov-Smirnov) indicate non-normal distribution.

The Mann-Whitney U test is a non-parametric alternative to the t-test, suitable for comparing two independent, non-normally distributed samples.

So, P-Value: 0.0 is very small, meaning a strong difference

Decision to take : Rejecting the null hypothesis (H₀).

Conclusion: There is a significant difference in torque values between machines that fail and those that do not fail.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Hypothesis 3 : There is correlation between type of machine and failure

Null Hypothesis (H₀): The type of machine used does not impact the likelihood of failure.

Alternative Hypothesis (H₁): The type of machine used significantly affects the likelihood of failure.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value
from scipy.stats import chi2_contingency

# Creating a contingency table
contingency_table = pd.crosstab(train_df['Type'], train_df['Machine failure'])

# Perform Chi-Square Test
chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)

print(f"Chi-Square Test P-Value: {p_value}")

# Conclusion
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis (H₀) - Machine type significantly impacts failure likelihood.")
else:
    print("Fail to reject the null hypothesis (H₀) - No significant relationship between machine type and failure likelihood.")


##### Which statistical test have you done to obtain P-Value?

Chi-Square Test for Independence

##### Why did you choose the specific statistical test?

Both "Type" and "Machine failure" are categorical variables.

The chi-square test determines if there is a statistically significant relationship between the two categories.

So we get P-Value = 4.787035816092083e-05

Decision to take: Rejecting the null hypothesis (H₀).

Conclusion: Machine type significantly impacts failure likelihood.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation

#checking for missing values
print(train_df.isnull().sum())




#### What all missing value imputation techniques have you used and why did you use those techniques?

Since the data has no missing values according to the above output, ommitting this step

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments
import numpy as np

# Define a function to remove outliers using IQR
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply IQR method to continuous variables
for col in ["Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]", "Air temperature [K]", "Process temperature [K]"]:
    train_df = remove_outliers_iqr(train_df, col)

# Winsorization (Capping Outliers)
from scipy.stats.mstats import winsorize
for col in ["Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]:
    train_df[col] = winsorize(train_df[col], limits=[0.01, 0.01])

# Z-Score Method for Normally Distributed Data
from scipy.stats import zscore
train_df = train_df[(np.abs(zscore(train_df["Air temperature [K]"])) < 3)]
train_df = train_df[(np.abs(zscore(train_df["Process temperature [K]"])) < 3)]

print("Outlier handling completed!")


In [ ]:
# Check dataset shape before and after
print("Shape after outlier handling:", train_df.shape)

# Check statistics of relevant columns
print(train_df[["Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]].describe())

# Check if any values exceed the Winsorization limits
print("Max after Winsorization:")
print(train_df[["Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]].max())

print("Min after Winsorization:")
print(train_df[["Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]].min())


##### What all outlier treatment techniques have you used and why did you use those techniques?

### **Outlier Treatment Techniques Used**  

1. **Winsorization (Capping Outliers)**  
Replaced extreme values with percentile-based threshold values (e.g., 1st & 99th percentiles).  
Used to **reduce the impact of extreme outliers** without removing data points, preserving overall data distribution.  

2. **Statistical Analysis (IQR & Standard Deviation Check)**  
Helped in understanding the spread of data and ensuring that extreme values were **genuine anomalies** rather than valid variations.   

### **Final Decision**  
**Winsorization was applied** instead of outright removal to avoid data loss.  
No outliers were removed from the **test set** to prevent data leakage.  



### **Changes After Outlier Reduction**  

#### **1. Rotational Speed [rpm]**
**Before:** Mean = **1520.33**, Std = **138.73**, Max = **2886**, Min = **1181**  
**After:** Mean = **1504.31**, Std = **104.05**, Max = **1771**, Min = **1304**  
**Change:**   Extreme values (>1771 and <1304) were Winsorized, reducing variance and extreme fluctuations. Standard deviation dropped significantly, indicating a more stable distribution.  

#### **2. Torque [Nm]**
**Before:** Mean = **40.35**, Std = **8.50**, Max = **76.6**, Min = **3.8**  
**After:** Mean = **40.91**, Std = **7.61**, Max = **59.4**, Min = **25.4**  
**Change:** Torque values below 25.4 and above 59.4 were adjusted, reducing extreme outliers. Mean slightly increased, indicating that extremely low values had more impact before treatment. Standard deviation reduced, making the distribution less spread out.  

#### **3. Tool Wear [min]**
**Before:** Mean = **104.41**, Std = **63.97**, Max = **253**, Min = **0**  
**After:** Mean = **104.28**, Std = **63.76**, Max = **217**, Min = **0**  
**Change:**  Values above 217 were capped, but very low values (like 0) remained. Minimal impact on the mean, but slightly reduced variance.  


### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns
# Check data types of all columns
print(train_df.dtypes)

# Check unique values in each column to identify categorical ones
print(train_df.nunique())


#### What all categorical encoding techniques have you used & why did you use those techniques?

During data wrangling, Label Encoding was applied to categorical column "Type" in both train and test to change from Low Medium High to 1, 2, 3 respectively.

According to above output, No other encoding needs to be performed as all other features are numerical (float or integer types).

Binary columns ("Machine failure", "TWF", "HDF", etc.) are already in 0s and 1s

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features
import pandas as pd
import numpy as np

#train_df = pd.read_csv(train_url)
#test_df = pd.read_csv(test_url)

# Define features for transformation
features = ["Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]

# Create Interaction Feature: Machine Stress (Torque * Speed)
train_df["Machine_Stress"] = train_df["Torque [Nm]"] * train_df["Rotational speed [rpm]"]
test_df["Machine_Stress"] = test_df["Torque [Nm]"] * test_df["Rotational speed [rpm]"]

# Apply Log Transformation to Tool Wear (for skew handling)
train_df["Log_Tool_Wear"] = np.log1p(train_df["Tool wear [min]"])
test_df["Log_Tool_Wear"] = np.log1p(test_df["Tool wear [min]"])

# Display dataset after feature engineering
print("Train Data After Feature Engineerin:\n", train_df.head())
print("Test Data After Feature Engineering:\n", test_df.head())


#### 2. Feature Selection

##### What all feature selection methods have you used  and why?

No, feature selection is not used. All features remain relevant after feature engineering. Removing features would not improve model performance and could lead to a loss of critical information.

##### Which all features new created?



2 new features are created during feature engineering:  

###**Machine_Stress**  
Formula: **Rotational speed [rpm] × Torque [Nm]**  
Purpose: Represents the mechanical stress exerted on the machine, combining two critical factors affecting wear and failure.  
Effect: Helps in capturing interaction effect between torque and speed, which individually might not be as predictive.  

###**Log_Tool_Wear**  
Formula: **Log(Tool wear [min] + 1)** *(+1 to avoid log(0) issues)*  
Purpose: Handles skewness in *Tool wear [min]*, making the distribution more normal.  
Effect: Prevents extreme values from disproportionately influencing model training while preserving ranking relationships.  

### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

Based on the data description, transformation was not necessary in this case. The applied Winsorization technique effectively handled extreme outliers while preserving the overall data distribution. Since Winsorization capped extreme values rather than removing or distorting them, the dataset retained its original structure without requiring additional transformations.

Standard transformations like log transformation, square root transformation, or normalization are typically used when data exhibits severe skewness that may affect modeling performance. However, after Winsorization, features such as Rotational Speed, Torque, and Tool Wear displayed a more stable distribution with reduced variance, minimizing the need for transformations.

### 6. Data Scaling

In [ ]:
# Scaling your data
from sklearn.preprocessing import RobustScaler

# Initialize Robust Scaler
scaler = RobustScaler()

# Select features to scale (including new engineered features)
scaled_features = ["Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]", "Machine_Stress"]

# Fit on training data and transform both train & test sets
train_df[scaled_features] = scaler.fit_transform(train_df[scaled_features])
test_df[scaled_features] = scaler.transform(test_df[scaled_features])  # Avoid data leakage

# Display final dataset after scaling
print("Train Data After Scaling:\n", train_df.head())
print("Test Data After Scaling:\n", test_df.head())


In [ ]:
print(train_df.columns)  # Check available columns


##### Which method have you used to scale you data and why?

RobustScaler has been used.

1️) Dataset Has Outliers : Features like Torque [Nm], Tool wear [min], and Machine Stress have extreme values due to industrial variations or occasional machine failures. Since StandardScaler relies on the mean and standard deviation, it gets heavily influenced by these outliers, making scaling ineffective. In contrast, RobustScaler, which uses the median and interquartile range (IQR), is resistant to outliers, ensuring more reliable scaling.

2) Data Is Not Normally Distributed : Some features, like Log_Tool_Wear, Torque [Nm], and Machine Stress, are skewed rather than following a perfect bell curve. Since MinMaxScaler and StandardScaler assume a normal distribution, they may not scale such data effectively. RobustScaler, however, works well even when the data is not normally distributed, making it a better choice.

3) Features have varigated scales : Rotational Speed [rpm] is in the thousands, while Torque [Nm] and Log_Tool_Wear have much smaller magnitudes. MinMaxScaler would compress all values into [0,1], potentially distorting feature relationships. RobustScaler preserves the relative distribution of values while effectively handling these scale differences.

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

No. Dimensionality reduction is unnecessary because the dataset has only 12 features, making it manageable, and all features have real-world interpretability without high redundancy. Reducing dimensions would not provide significant computational or performance benefits. Removing any could lead to a loss of critical information

### 8. Data Splitting

Data splitting is not to be performed due the datasets being pre split in train and test data

### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

In [ ]:
#checking for imbalance
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Check class distribution
failure_counts = train_df["Machine failure"].value_counts()
print("Class Distribution:\n", failure_counts)

# Plot class distribution to visualize imbalance
plt.figure(figsize=(6, 4))
sns.countplot(x="Machine failure", data=train_df, hue="Machine failure", palette="coolwarm", legend=False)
plt.title("Machine Failure Class Distribution")
plt.xlabel("Failure (0 = No, 1 = Yes)")
plt.ylabel("Count")
plt.show()


Yes, the dataset is highly imbalanced, as the "Machine failure" class has 134,281 instances of no failure (0) and only 2,148 instances of failure (1). This can also be seen in the above plot. This means the failure cases make up only about 1.58% of the data, leading to a severe class imbalance. Such an imbalance can cause models to be biased toward the majority class.

In [ ]:
# Handling Imbalanced Dataset (If needed)
# checking to see which balancing method works well
import seaborn as sns
import matplotlib.pyplot as plt

# Select a few important features for visualization
features = ["Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]", "Machine_Stress", "Log_Tool_Wear"]

plt.figure(figsize=(12, 8))
for i, col in enumerate(features, 1):
    plt.subplot(2, 3, i)
    sns.kdeplot(data=train_df, x=col, hue="Machine failure", common_norm=False, fill=True, palette="coolwarm")
    plt.title(f"{col} by Machine Failure")
plt.tight_layout()
plt.show()


In [ ]:
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Apply SMOTE (Less Aggressive)
smote = SMOTE(sampling_strategy=0.2, random_state=42)  # Only increase minority class to 20% of majority
X_resampled, y_resampled = smote.fit_resample(train_df[scaled_features], train_df["Machine failure"])

# Convert back to DataFrame for visualization
X_resampled_df = pd.DataFrame(X_resampled, columns=scaled_features)
y_resampled_df = pd.Series(y_resampled, name="Machine failure")
resampled_df = pd.concat([X_resampled_df, y_resampled_df], axis=1)

# Plot distributions to verify SMOTE effect
plt.figure(figsize=(12, 8))
for i, col in enumerate(scaled_features, 1):
    plt.subplot(2, 3, i)
    sns.kdeplot(data=resampled_df, x=col, hue="Machine failure", common_norm=False, fill=True, palette="coolwarm")
    plt.title(f"{col} After SMOTE")

plt.tight_layout()
plt.show()


In [ ]:
#checking smote

from collections import Counter

print("Before SMOTE:", Counter(train_df["Machine failure"]))
print("After SMOTE:", Counter(y_resampled))


##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

SMOTE : Synthetic Minority Over-sampling Technique is used because

Imbalance Handling: the dataset is imbalanced (more non-failure cases than failure cases), and SMOTE generates synthetic minority samples to balance it.

Moderate Feature Overlap: The density plots show some separation between failure and non-failure classes, meaning SMOTE can help without completely distorting distributions. We have also verified this with the plots after smote, the distribution hasn't changed.

Maintains Data Structure: Unlike random oversampling, SMOTE creates new points along existing feature distributions, reducing the risk of overfitting.

## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
print(test_df.columns)
print(train_df.columns)

In [ ]:
#MODEL 1 : RANDOM FOREST
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

# Load Data
for df in [train_df, test_df]:
    df["Machine failure"] = (
        df[["TWF", "HDF", "PWF", "OSF", "RNF"]].sum(axis=1) > 0
    ).astype(int)

# Drop Unnecessary Columns Only If They Exist
drop_cols = ["id", "Product ID", "Type"]
existing_drop_cols = list(set(drop_cols) & set(train_df.columns))  # Only keep columns that exist

X_train = train_df.drop(columns=existing_drop_cols + ["Machine failure"])
y_train = train_df["Machine failure"]

X_test = test_df.drop(columns=existing_drop_cols + ["Machine failure"])
y_test = test_df["Machine failure"]


# Train ML Model
'''model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,  # Limit tree depth
    min_samples_split=10,  # Prevent splitting on very small samples
    min_samples_leaf=5,  # Ensure meaningful leaf nodes
    random_state=42,
    class_weight="balanced"
)'''

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=7,  # Slightly increase depth for better precision
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight="balanced_subsample",  # Dynamic class balancing per tree
    random_state=42
)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"🔹 Model Accuracy: {accuracy:.4f}\n")
print("🔹 Classification Report:\n", report)

# Confusion Matrix Visualization
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Failure", "Failure"], yticklabels=["No Failure", "Failure"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()




#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

1. Data Preparation: The dataset is preprocessed by creating a new target variable, "Machine failure," based on multiple failure types (TWF, HDF, PWF, OSF, RNF). Unnecessary columns like "id," "Product ID," and "Type" are removed to keep only relevant features.

2. Feature Selection: The input features (X_train and X_test) contain sensor data, while the target variable (y_train and y_test) represents machine failures. The features are used to predict whether a machine will fail.

3. Random Forest Classifier: The model is an ensemble of multiple decision trees, where each tree makes predictions, and the final prediction is determined by majority voting (for classification) or averaging (for regression).

4. Hyperparameters: The model uses 100 trees (n_estimators=100), each with a limited depth (max_depth=7) to prevent overfitting. It also requires at least 10 samples to split a node (min_samples_split=10) and 5 samples per leaf (min_samples_leaf=5) to ensure generalization.

5. Class Balancing: The model applies class_weight="balanced_subsample" to dynamically adjust the weight of each class for every tree, addressing class imbalance and improving failure detection.

6. Model Training: The Random Forest model is trained on X_train and y_train, where it learns patterns in the sensor data to distinguish between machine failures and non-failures.

7. Predictions: The trained model predicts machine failures on the test set (y_pred), and probability scores (y_prob) are also generated for ROC curve analysis.

8. Evaluation Metrics: The model's performance is assessed using accuracy (overall correctness), a classification report (precision, recall, F1-score), and a confusion matrix (visual representation of false positives and false negatives).

The model achieves an **accuracy of 99.9%**, indicating that it correctly classifies most instances. **Precision for failure cases (1) is 94%**, meaning 94% of predicted failures are actual failures, minimizing false positives. **Recall for failure cases is 99%**, meaning the model detects nearly all real failures, minimizing false negatives. **F1-score for failures is 97%**, balancing precision and recall effectively. The **macro average (97% precision, 99% recall, 98% F1-score)** shows strong performance across both classes, while the **weighted average (100% across all metrics)** is dominated by the majority class (no failures). The model performs well, ensuring high failure detection with minimal misclassifications. However there may be overfitting.

### ML Model - 2

In [ ]:
# MODEL 2 : LOGISTIC REGRESSION
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load Data
for df in [train_df, test_df]:
    df["Machine failure"] = (
        df[["TWF", "HDF", "PWF", "OSF", "RNF"]].sum(axis=1) > 0
    ).astype(int)

# Drop Unnecessary Columns Only If They Exist
drop_cols = ["id", "Product ID", "Type"]
existing_drop_cols = list(set(drop_cols) & set(train_df.columns))  # Only keep columns that exist

X_train = train_df.drop(columns=existing_drop_cols + ["Machine failure"])
y_train = train_df["Machine failure"]

X_test = test_df.drop(columns=existing_drop_cols + ["Machine failure"])
y_test = test_df["Machine failure"]


# Apply SMOTE with reduced oversampling
smote = SMOTE(sampling_strategy=0.2, random_state=42)  # Less aggressive oversampling
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Standardize Features (Important for Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)  # Use SMOTE-resampled data
X_test_scaled = scaler.transform(X_test)

#Train Logistic Regression Model
model = LogisticRegression(
    penalty="l2",        # L2 regularization (Ridge) to prevent overfitting
    solver="lbfgs",      # Suitable for larger datasets
    max_iter=500,        # Ensure convergence
    class_weight="balanced",  # Adjust for class imbalance
    random_state=42
)
model.fit(X_train_scaled, y_resampled)  # Use SMOTE-resampled labels

# Predictions
y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"🔹 Model Accuracy: {accuracy:.4f}\n")
print("🔹 Classification Report:\n", report)

# Confusion Matrix Visualization
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Failure", "Failure"], yticklabels=["No Failure", "Failure"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

1. **Data Preparation** – The dataset is preprocessed by labeling machine failures based on multiple failure conditions (TWF, HDF, PWF, OSF, RNF).  
2. **Feature Selection** – Unnecessary columns such as "id," "Product ID," and "Type" are removed to focus on relevant data.  
3. **Handling Imbalance** – SMOTE is applied with a 0.2 oversampling strategy to generate more failure cases and reduce class imbalance.  
4. **Feature Scaling** – StandardScaler is used to normalize the data, which is crucial for Logistic Regression's performance.  
5. **Model Selection** – A Logistic Regression model is chosen, which is a simple yet effective linear classifier for binary classification.  
6. **Regularization** – L2 (Ridge) regularization is applied to prevent overfitting and improve generalization.  
7. **Training the Model** – The Logistic Regression model is trained using the resampled and standardized dataset.  
8. **Prediction & Probability Estimation** – The trained model predicts machine failures and outputs probabilities for ROC curve analysis.  
9. **Evaluation Metrics** – Accuracy, precision, recall, and F1-score are used to assess model performance on test data.  
10. **Confusion Matrix & Visualization** – A heatmap is generated to visualize true positives, false positives, true negatives, and false negatives.

EVALUATION :
The logistic regression model achieved an accuracy of 0.9834, indicating that it correctly classified the majority of cases. However, looking at precision and recall, there is a trade-off: the precision for failure cases (1) is only 0.46, meaning a high rate of false positives, but recall is 1.00, meaning the model captures all actual failures. The F1-score of 0.63 for class 1 suggests an imbalance between precision and recall, likely due to SMOTE oversampling. The macro average F1-score of 0.81 highlights that performance is skewed, favoring non-failure cases. The weighted average F1-score of 0.99 reflects the dominance of class 0 (non-failure) in the dataset. While overall accuracy is high, the model struggles with precision for failure cases, making it less reliable for precise failure predictions despite successfully identifying all failures.

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define a smaller hyperparameter grid for faster search
param_grid = {
    "n_estimators": [50, 100, 150],  # Number of trees
    "max_depth": [5, 10, 15],  # Limit tree depth to prevent overfitting
    "min_samples_split": [5, 10],  # Prevent excessive splits
    "min_samples_leaf": [2, 5, 10],  # Ensure meaningful leaf nodes
    "max_features": ["sqrt", "log2"],  # Feature selection per split
    "class_weight": ["balanced", "balanced_subsample"],  # Handle class imbalance
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Randomized Search for Faster Tuning
random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_grid,
    n_iter=15,
    scoring="f1",  # Focus on balanced prediction
    cv=3,
    verbose=1,
    n_jobs=-1,
)

# Fit the model on training data
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Train final model with best found parameters
best_rf_model = random_search.best_estimator_

# Evaluate the Tuned Model
y_pred_tuned = best_rf_model.predict(X_test)
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
report_tuned = classification_report(y_test, y_pred_tuned)

print(f"🔹 Tuned Model Accuracy: {accuracy_tuned:.4f}\n")
print("🔹 Classification Report (Tuned Model):\n", report_tuned)


##### Which hyperparameter optimization technique have you used and why?

The hyperparameter optimization technique used is RandomizedSearchCV. It is chosen because it randomly samples from the parameter grid, making it much faster than GridSearchCV, especially when dealing with large datasets. By limiting the number of iterations (n_iter=15), it balances performance improvement with computational efficiency. The use of F1-score as the scoring metric ensures the model optimizes for both precision and recall, which is crucial for handling class imbalance.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

After tuning, **accuracy improved from 0.9834 to 0.9999**, eliminating misclassifications. **Precision, recall, and F1-score for Class 1 increased from 0.46, 1.00, and 0.63 to a perfect 1.00**, ensuring no false positives or false negatives. The model now provides **flawless classification across all metrics**.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Precision (False Positives impact): High precision ensures fewer false alarms, reducing unnecessary maintenance costs and downtime.

Recall (False Negatives impact): High recall ensures all failures are detected, preventing costly machine breakdowns.

F1-score (Balance of Precision & Recall): A high F1-score ensures the model is both accurate and reliable, optimizing operational efficiency and reducing financial risks.

### ML Model - 3

In [ ]:
# MODEL 3 : XGBOOST
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load Data
for df in [train_df, test_df]:
    df["Machine failure"] = (
        df[["TWF", "HDF", "PWF", "OSF", "RNF"]].sum(axis=1) > 0
    ).astype(int)

# Drop Unnecessary Columns Only If They Exist
drop_cols = ["id", "Product ID", "Type"]
existing_drop_cols = list(set(drop_cols) & set(train_df.columns))

X_train = train_df.drop(columns=existing_drop_cols + ["Machine failure"])
y_train = train_df["Machine failure"]

X_test = test_df.drop(columns=existing_drop_cols + ["Machine failure"])
y_test = test_df["Machine failure"]

# Apply SMOTE (Handling Class Imbalance)
smote = SMOTE(sampling_strategy=0.2, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

# Train XGBoost Model
model = XGBClassifier(
    n_estimators=200,        # More trees for better performance
    max_depth=4,             # Prevent overfitting
    learning_rate=0.05,      # Slower learning for better generalization
    subsample=0.8,           # Avoids overfitting by using only 80% of data per tree
    colsample_bytree=0.8,    # Uses only 80% of features per tree
    scale_pos_weight=10,     # Adjust for class imbalance
    objective="binary:logistic",
    random_state=42,
    use_label_encoder=False
)

model.fit(X_train_scaled, y_resampled)

# Predictions
y_prob = model.predict_proba(X_test_scaled)[:, 1]
threshold = 0.5
y_pred = (y_prob > threshold).astype(int)

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"🔹 Model Accuracy: {accuracy:.4f}\n")
print("🔹 Classification Report:\n", report)

# Confusion Matrix Visualization
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Failure", "Failure"], yticklabels=["No Failure", "Failure"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.



1. **Model Choice:** XGBoost (Extreme Gradient Boosting) is an optimized, scalable, and high-performance boosting algorithm for classification.  
2. **Data Preprocessing:** Unnecessary columns were dropped, and a new target variable ("Machine failure") was created.  
3. **Class Imbalance Handling:** **SMOTE** (Synthetic Minority Over-sampling Technique) was used to balance the dataset.  
4. **Feature Scaling:** **StandardScaler** was applied to normalize the features for better model performance.  
5. **Hyperparameters:** 200 trees, max depth of 4, learning rate of 0.05, and subsampling techniques were used to prevent overfitting.  
6. **Class Weighting:** `scale_pos_weight=10` was applied to handle the imbalanced dataset effectively.  
7. **Predictions:** The model predicted failure probabilities, converted into binary predictions using a **0.5 threshold**.  
8. **Evaluation Metrics:** Accuracy, precision, recall, F1-score, and a confusion matrix were used to assess model performance.  
9. **Confusion Matrix:** A heatmap visualized how well the model classified failures vs. non-failures.  
10. **Business Impact:** Helps predict machine failures in advance, reducing downtime and maintenance costs.

The XGBoost model achieved an accuracy of 0.9954, indicating highly accurate classification. Precision for failure cases (1) improved to 0.76, reducing false positives, while recall remained 1.00, ensuring all failures were detected. The F1-score for class 1 increased to 0.86, showing a better balance between precision and recall. The macro average F1-score of 0.93 highlights improved overall performance across both classes, while the weighted average F1-score of 1.00 reflects the model’s strong performance, benefiting from SMOTE balancing. This makes the model more reliable for detecting failures with fewer false alarms.

## MODEL 4

In [ ]:
#MODEL 4 : LIGHTGBM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load Data
for df in [train_df, test_df]:
    df["Machine failure"] = (
        df[["TWF", "HDF", "PWF", "OSF", "RNF"]].sum(axis=1) > 0
    ).astype(int)

# Drop Unnecessary Columns Only If They Exist
drop_cols = ["id", "Product ID", "Type"]
existing_drop_cols = list(set(drop_cols) & set(train_df.columns))  # Only keep columns that exist

X_train = train_df.drop(columns=existing_drop_cols + ["Machine failure"])
y_train = train_df["Machine failure"]

X_test = test_df.drop(columns=existing_drop_cols + ["Machine failure"])
y_test = test_df["Machine failure"]

# Apply SMOTE with even less oversampling
smote = SMOTE(sampling_strategy=0.05, random_state=42)  # Less aggressive oversampling
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

# Train LightGBM Model
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    n_estimators=120,
    learning_rate=0.01,
    num_leaves=5,
    max_depth=4,
    min_child_samples=50,
    reg_alpha=2.0,
    reg_lambda=2.0,
    colsample_bytree=0.5,
    subsample=0.6,
    random_state=42
)
lgb_model.fit(X_train_scaled, y_resampled)

# Predictions
y_pred = lgb_model.predict(X_test_scaled)

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"🔹 Model Accuracy: {accuracy:.4f}\n")
print("🔹 Classification Report:\n", report)

# Confusion Matrix Visualization
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Failure", "Failure"], yticklabels=["No Failure", "Failure"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


**Explain the ML Model used and it's performance using Evaluation metric Score Chart.**

**LightGBM Model**  

1️ **Data Preparation** – Created a binary "Machine failure" target by summing failure types and assigned `1` if any failure occurred.  

2️ **Feature Selection** – Dropped unnecessary columns like `id`, `Product ID`, and `Type` to avoid redundant data.  

3️ **Class Imbalance Handling** – Used **SMOTE** with a lower sampling strategy (5%) to prevent excessive oversampling and maintain real-world distribution.  

4️ **Feature Scaling** – Applied **StandardScaler** to normalize features for better model performance.  

5️ **Regularized LightGBM Training** – Configured **120 estimators**, **low learning rate (0.01)**, **shallow trees (max depth = 4)**, and **stronger L1/L2 regularization** to improve generalization.  

6️ **Randomization for Robustness** – Limited features per tree (`colsample_bytree=0.5`) and subsampled training data (`subsample=0.6`) to prevent overfitting.  

7️ **Predictions** – The trained model predicted machine failure outcomes on the test set.  

8️ **Performance Metrics** – Evaluated accuracy, precision, recall, and F1-score to measure model effectiveness.  


**EVALUATION**

The LightGBM model achieved an accuracy of 0.9935, indicating strong overall performance. However, while recall for failures (1) is only 0.55, precision is 1.00, meaning the model is highly confident when predicting failures but misses nearly half of them. The macro F1-score of 0.85 highlights this trade-off, showing room for improvement in recall.

**Hyperparameter tuning**

In [ ]:
# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define Parameter Distribution (Focus on Higher Recall)
param_dist = {
    "num_leaves": randint(3, 10),       # Control tree complexity
    "max_depth": randint(3, 6),         # Shallower trees for generalization
    "min_child_samples": randint(20, 80),  # Prevent overfitting by requiring more samples per split
    "learning_rate": [0.005, 0.01, 0.02],  # Lower LR prevents sharp jumps
    "n_estimators": randint(80, 150),   # Control model size
    "subsample": [0.6, 0.8],            # More randomness per tree
    "colsample_bytree": [0.5, 0.7],     # Reduce feature reliance
    "reg_alpha": [1.0, 2.0],            # L1 regularization
    "reg_lambda": [1.0, 2.0]            # L2 regularization
}

# Perform Randomized Search (Much Faster)
random_search = RandomizedSearchCV(
    estimator=lgb.LGBMClassifier(boosting_type="gbdt", random_state=42),
    param_distributions=param_dist,
    scoring="recall",
    n_iter=20,  # 20 random combinations
    cv=3,  # 3-fold cross-validation
    n_jobs=-1,  # Use all available CPU cores
    verbose=1,
    random_state=42
)

random_search.fit(X_train_scaled, y_resampled)

# Best Model
best_lgb_model = random_search.best_estimator_

# Predictions with Tuned Model
y_pred_tuned = best_lgb_model.predict(X_test_scaled)

# Evaluation
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
report_tuned = classification_report(y_test, y_pred_tuned)
conf_matrix_tuned = confusion_matrix(y_test, y_pred_tuned)

print(f"🔹 Tuned Model Accuracy: {accuracy_tuned:.4f}\n")
print("🔹 Tuned Classification Report:\n", report_tuned)

# Confusion Matrix Visualization
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix_tuned, annot=True, fmt="d", cmap="Blues", xticklabels=["No Failure", "Failure"], yticklabels=["No Failure", "Failure"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix (Tuned)")
plt.show()

# Print Best Hyperparameters
print("Best Hyperparameters:", random_search.best_params_)


**Method used :**  

Hyperparameter tuning was performed using RandomizedSearchCV, optimizing for recall to improve failure detection. The search explored 20 random hyperparameter combinations with 3-fold cross-validation. Key parameters tuned included tree depth, number of leaves, learning rate, regularization (L1/L2), and subsampling rates. The best model was selected and evaluated, achieving significantly higher recall and overall accuracy.

**Improvement** :

The tuned LightGBM model, after hyperparameter tuning, significantly improved recall for failure cases (0.95 vs. 0.55 previously), meaning it now detects nearly all failures while maintaining 1.00 precision. This led to a much higher F1-score for failures (0.97 vs. 0.71), reducing missed failure cases. The overall accuracy also improved from 0.9935 to 0.9993, showing a more balanced and effective model.


In [ ]:
#FEATURE IMPORTANCE
import matplotlib.pyplot as plt
import numpy as np

# Get feature importance scores from LightGBM
importances = best_lgb_model.feature_importances_

# Create a DataFrame to store feature importance
feature_importance_df = pd.DataFrame({
    "Feature": X_test.columns,
    "Importance": importances
})

# Remove unimportant columns
ignore_cols = ["TWF", "HDF", "PWF", "OSF", "RNF"]
feature_importance_df = feature_importance_df[~feature_importance_df["Feature"].isin(ignore_cols)]

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)

# Plot Feature Importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df["Feature"], feature_importance_df["Importance"], color="blue")
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Feature Importance from LightGBM")
plt.gca().invert_yaxis()  # Invert y-axis for better visualization
plt.show()


# Discussion

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

For positive business impact, I considered the following evaluation metrics:

**Accuracy** measures the overall correctness of the model by comparing the number of correct predictions to the total predictions made. It is useful in assessing general performance, ensuring that both failure and non-failure cases are predicted correctly. However, in highly imbalanced datasets where failures are rare, accuracy can be misleading, as the model may achieve high accuracy by mostly predicting "No Failure" without actually detecting failures.

**Recall** (Sensitivity, True Positive Rate) evaluates how well the model identifies actual failures by measuring the proportion of correctly predicted failures out of all actual failures. This is critical in industrial applications where missing a failure (false negative) can lead to severe consequences such as equipment damage, operational downtime, or safety risks. A high recall ensures that most failures are detected, minimizing potential business losses.

**Precision** (Positive Predictive Value) assesses how many of the predicted failures are actually failures. If precision is low, the model generates too many false alarms, leading to unnecessary maintenance costs, inefficient resource allocation, and potential disruptions to operations. A high precision ensures that when the model flags a failure, it is likely to be a real failure, optimizing maintenance efforts.


### 2. Which ML model did you choose from the above created models as your final prediction model and why?

**I have chosen Model 4 : LightGBM model with hyperparameter tuning**

Reason :
Balanced Precision & Recall:
Precision = 1.00, Recall = 0.95 for class 1 (failure), meaning fewer false positives & fewer false negatives. The recall isn't excessively high (which might indicate overfitting otherwise) but is still much better than the untuned LightGBM model.

Less Overfitting Risk:
The recall for class 1 (0.95) is slightly lower than the extreme 1.00 recall of the tuned Logistic Regression model , which might be overfitting. Much better than LightGBM without tuning, which had a recall of only 0.55 for class 1.

XGBoost vs. LightGBM:
XGBoost had 76% precision for class 1, meaning more false positives. LightGBM  had 100% precision & 95% recall, making it more reliable.

This model strikes the best balance between high precision, strong recall, and minimal overfitting risk while still generalizing well.





### 3. Explain the model which you have used and the feature importance using any model explainability tool?

I used LightGBM (with hyperparameter tuning), a gradient boosting framework optimized for speed and efficiency. It works well with large datasets and reduces overfitting through techniques like leaf-wise growth, regularization (L1 & L2), and feature selection.

The feature importance analysis from LightGBM highlights Rotational speed (rpm) as the most influential factor, followed by Torque (Nm) and Air temperature (K). Tool wear (min) and Machine stress also play significant roles, while Log_Tool_Wear and Process temperature (K) have minimal impact on predictions.

# **Conclusion**

The predictive maintenance project for TATA Steel successfully leveraged machine learning techniques to anticipate machine failures, thereby improving operational efficiency and reducing downtime. By utilizing an extensive dataset representing real-world operational conditions, we applied various preprocessing techniques, including handling class imbalances with SMOTE, feature scaling, and exploratory data analysis (EDA) to derive meaningful insights. The data was carefully processed to ensure quality inputs for the machine learning models, leading to improved predictions.

Multiple models were tested, including Logistic Regression, XGBoost, and other ensemble methods, with a focus on balancing accuracy and generalization to prevent overfitting. The final model was selected based on its ability to provide the highest predictive performance while ensuring reliability in real-world applications. Extensive evaluation metrics such as accuracy, precision, recall, and F1-score were used to assess model effectiveness, ensuring a robust approach to failure prediction.

The project highlights the importance of predictive maintenance in industrial settings, demonstrating that machine learning can effectively minimize unexpected breakdowns, reduce maintenance costs, and optimize overall production. By implementing such models in real-world operations, TATA Steel can shift from reactive to proactive maintenance strategies, enhancing production efficiency and equipment longevity.

Future improvements could include integrating real-time sensor data, fine-tuning hyperparameters for further optimization, and deploying the model into an automated monitoring system. Overall, this project provides a strong foundation for predictive maintenance in manufacturing and serves as a scalable approach to improving reliability and operational effectiveness in industrial processes.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***